In [3]:
import pandas as pd
import numpy as np

In [4]:

from collections import defaultdict, Counter
from itertools import combinations

In [5]:
transactions = [
    ['bread', 'milk', 'eggs'],
    ['bread', 'diapers', 'beer', 'cola'],
    ['milk', 'diapers', 'beer', 'chips'],
    ['bread', 'milk', 'diapers', 'beer'],
    ['bread', 'milk', 'eggs', 'cola'],
    ['bread', 'diapers', 'milk'],
    ['beer', 'diapers', 'eggs'],
    ['bread', 'beer', 'chips'],
    ['bread', 'milk', 'diapers'],
    ['milk', 'eggs', 'cola']
]


In [6]:
def calculate_item_frequencies(transactions):
   
    item_freq = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            item_freq[item] += 1
    return item_freq


In [7]:
freq = calculate_item_frequencies(transactions)
print(freq)

defaultdict(<class 'int'>, {'bread': 7, 'milk': 7, 'eggs': 4, 'diapers': 6, 'beer': 5, 'cola': 3, 'chips': 2})


In [8]:
def filter_transactions(transactions, item_frequencies, min_support=2):
    # Identify frequent items
    frequent_items = {
        item for item, count in item_frequencies.items() 
        if count >= min_support
    }
    
    # Sort transactions by item frequency
    def sort_key(item):
        return item_frequencies.get(item, 0)
    
    filtered_transactions = []
    for transaction in transactions:
        # Filter and sort items within each transaction
        sorted_trans = sorted(
            [item for item in transaction if item in frequent_items], 
            key=sort_key, 
            reverse=True
        )
        if sorted_trans:
            filtered_transactions.append(sorted_trans)
    
    return filtered_transactions


In [9]:
filtered = filter_transactions(transactions, freq, min_support=2)
print(filtered)

[['bread', 'milk', 'eggs'], ['bread', 'diapers', 'beer', 'cola'], ['milk', 'diapers', 'beer', 'chips'], ['bread', 'milk', 'diapers', 'beer'], ['bread', 'milk', 'eggs', 'cola'], ['bread', 'milk', 'diapers'], ['diapers', 'beer', 'eggs'], ['bread', 'beer', 'chips'], ['bread', 'milk', 'diapers'], ['milk', 'eggs', 'cola']]


In [ ]:

def construct_fp_tree(transactions):
    # Calculate Item Frequencies
    item_frequencies = {}
    for transaction in transactions:
        for item in transaction:
            item_frequencies[item] = item_frequencies.get(item, 0) + 1
    
    print("Step 1: Item Frequencies")
    print(item_frequencies)
    print("\n")
    
    # Sort and Filter Transactions
    def sort_transaction(transaction):
        return sorted(transaction, key=lambda x: item_frequencies.get(x, 0), reverse=True)
    
    sorted_transactions = [sort_transaction(trans) for trans in transactions]
    
    print("Step 2: Sorted Transactions")
    for i, trans in enumerate(sorted_transactions, 1):
        print(f"T{i}: {trans}")
    print("\n" )
    
    # Initialize FP-Tree
    fp_tree = {
        'root': {
            'item': None,
            'count': 0,
            'children': {},
            'parent': None
        }
    }
    header_table = {}
    
    # Build FP-Tree
    tree_building_steps = []
    
    for transaction_num, transaction in enumerate(sorted_transactions, 1):
        current_node = fp_tree['root']
        path = []
        
        for item in transaction:
            # If item not in children, create new node
            if item not in current_node['children']:
                new_node = {
                    'item': item,
                    'count': 1,
                    'children': {},
                    'parent': current_node
                }
                current_node['children'][item] = new_node
                
                # Update header table
                if item not in header_table:
                    header_table[item] = new_node
                
                # Track tree building step
                path.append(f"T{transaction_num}: Added {item} (New Node)")
            else:
                # Increment count of existing node
                current_node['children'][item]['count'] += 1
                path.append(f"T{transaction_num}: Incremented {item}")
            
            # Move to current item's node
            current_node = current_node['children'][item]
        
        # Store building steps for this transaction
        tree_building_steps.append(path)
    
    #Print Tree Building Steps
    print("Step 3: Tree Building Steps")
    for i, steps in enumerate(tree_building_steps, 1):
        print(f"Transaction {i} Steps:")
        for step in steps:
            print(f"  {step}")
            print()
        print()
    print("\n")
    
    print("Step 4: Tree Structure Analysis")
    def analyze_tree(node, level=0):
        indent = "  " * level
        if node['item'] is not None:
            print(f"{indent}{node['item']} (Count: {node['count']})")
        
        for child in node['children'].values():
            analyze_tree(child, level + 1)
    
    print("Tree Hierarchy:")
    analyze_tree(fp_tree['root'])
    print("\n")
    
    print("Step 5: Header Table")
    print("Items in Header Table:")
    for item, node in header_table.items():
        print(f"{item}: First Node Count = {node['count']}")
        print()
    
    return {
        'fp_tree': fp_tree,
        'header_table': header_table,
        'item_frequencies': item_frequencies,
        'transactions': transactions
    }

In [17]:
fpt = construct_fp_tree(filtered)
print(fpt)

Step 1: Item Frequencies
{'bread': 7, 'milk': 7, 'eggs': 4, 'diapers': 6, 'beer': 5, 'cola': 3, 'chips': 2}

Step 2: Sorted Transactions
T1: ['bread', 'milk', 'eggs']
T2: ['bread', 'diapers', 'beer', 'cola']
T3: ['milk', 'diapers', 'beer', 'chips']
T4: ['bread', 'milk', 'diapers', 'beer']
T5: ['bread', 'milk', 'eggs', 'cola']
T6: ['bread', 'milk', 'diapers']
T7: ['diapers', 'beer', 'eggs']
T8: ['bread', 'beer', 'chips']
T9: ['bread', 'milk', 'diapers']
T10: ['milk', 'eggs', 'cola']

Step 3: Tree Building Steps
Transaction 1 Steps:
  T1: Added bread (New Node)
  T1: Added milk (New Node)
  T1: Added eggs (New Node)

Transaction 2 Steps:
  T2: Incremented bread
  T2: Added diapers (New Node)
  T2: Added beer (New Node)
  T2: Added cola (New Node)

Transaction 3 Steps:
  T3: Added milk (New Node)
  T3: Added diapers (New Node)
  T3: Added beer (New Node)
  T3: Added chips (New Node)

Transaction 4 Steps:
  T4: Incremented bread
  T4: Incremented milk
  T4: Added diapers (New Node)
  T4: A

In [12]:
def find_frequent_itemsets(filtered_transactions, item_frequencies, min_support=2):
   
    frequent_itemsets = {}
    
    for item, count in item_frequencies.items():
        if count >= min_support:
            frequent_itemsets[frozenset([item])] = count
    
    return frequent_itemsets
print(find_frequent_itemsets(filtered, freq, min_support=2))

{frozenset({'bread'}): 7, frozenset({'milk'}): 7, frozenset({'eggs'}): 4, frozenset({'diapers'}): 6, frozenset({'beer'}): 5, frozenset({'cola'}): 3, frozenset({'chips'}): 2}


In [ ]:
def fp_tree_analysis(transactions, min_support=2):

    item_frequencies = calculate_item_frequencies(transactions)
    
    filtered_transactions = filter_transactions(
        transactions, 
        item_frequencies, 
        min_support
    )
    
    # Construct FP-Tree
    fp_tree, header_table = construct_fp_tree(filtered_transactions)
    
    # Find frequent itemsets
    frequent_itemsets = find_frequent_itemsets(
        filtered_transactions, 
        item_frequencies, 
        min_support
    )
    
    return {
        'item_frequencies': item_frequencies,
        'filtered_transactions': filtered_transactions,
        'frequent_itemsets': frequent_itemsets,
        'fp_tree': fp_tree
    }

print(fp_tree_analysis(transactions, min_support=2))

{'item_frequencies': defaultdict(<class 'int'>, {'bread': 7, 'milk': 7, 'eggs': 4, 'diapers': 6, 'beer': 5, 'cola': 3, 'chips': 2}), 'filtered_transactions': [['bread', 'milk', 'eggs'], ['bread', 'diapers', 'beer', 'cola'], ['milk', 'diapers', 'beer', 'chips'], ['bread', 'milk', 'diapers', 'beer'], ['bread', 'milk', 'eggs', 'cola'], ['bread', 'milk', 'diapers'], ['diapers', 'beer', 'eggs'], ['bread', 'beer', 'chips'], ['bread', 'milk', 'diapers'], ['milk', 'eggs', 'cola']], 'frequent_itemsets': {frozenset({'bread'}): 7, frozenset({'milk'}): 7, frozenset({'eggs'}): 4, frozenset({'diapers'}): 6, frozenset({'beer'}): 5, frozenset({'cola'}): 3, frozenset({'chips'}): 2}, 'fp_tree': {'children': {'bread': {'item': 'bread', 'count': 7, 'children': {'milk': {'item': 'milk', 'count': 5, 'children': {'eggs': {'item': 'eggs', 'count': 2, 'children': {'cola': {'item': 'cola', 'count': 1, 'children': {}, 'parent': {...}}}, 'parent': {...}}, 'diapers': {'item': 'diapers', 'count': 3, 'children': {'b

In [14]:
analysis_results = fp_tree_analysis(transactions, min_support=2)

print("Item Frequencies:")
for item, freq in analysis_results['item_frequencies'].items():
    print(f"{item}: {freq}")

print("\nFrequent Itemsets:")
for itemset, support in analysis_results['frequent_itemsets'].items():
    print(f"{list(itemset)}: {support}")

# a) Maximum Frequent Itemset
max_frequent_itemset = max(
    analysis_results['frequent_itemsets'], 
    key=lambda x: analysis_results['frequent_itemsets'][x]
)
print(f"\nMaximum Frequent Itemset: {list(max_frequent_itemset)} " +
      f"(Support: {analysis_results['frequent_itemsets'][max_frequent_itemset]})")

# b) Number of Transactions
print(f"\nTotal Transactions: {len(transactions)}")


Item Frequencies:
bread: 7
milk: 7
eggs: 4
diapers: 6
beer: 5
cola: 3
chips: 2

Frequent Itemsets:
['bread']: 7
['milk']: 7
['eggs']: 4
['diapers']: 6
['beer']: 5
['cola']: 3
['chips']: 2

Maximum Frequent Itemset: ['bread'] (Support: 7)

Total Transactions: 10
